# Global Historical Climate Network data in BigQuery

Historical daily weather data from the Global Historical Climate Network (GHCN) is available in BigQuery. Data comes from 80,000 stations in 180 countries. It has been observed that weather can impact up to 3.4% of the GDP. Find more at [https://cloud.google.com/blog/products/gcp/global-historical-daily-weather-data-now-available-in-bigquery](https://cloud.google.com/blog/products/gcp/global-historical-daily-weather-data-now-available-in-bigquery).

In [1]:
import os
import sys
import gmplot
import pandas as pd
from dynaconf import settings
import datalab.bigquery as bq

In [2]:
# add the path of the helpers package so we can import its functions e.g. import utils
path_helpers_package = os.path.join(settings.APP_PATH, "data2day_2022/helpers")
sys.path.append(path_helpers_package)

In [3]:
import utils

## Data Analysis

The first step is to identify the clostest GHCN station from Stuttgart.

After a quick tour on Google Map, we find out our latitude is 48.7751016 degrees latitude and your longitude is 9.1799984 degrees longitude.

In [4]:
# retrieve the string sql query
sql = utils.read_query("../data2day_2022/sql/find_stations_close_to.sql")

In [5]:
# parametrise the query and fetch the public BigQuery dataset
stations = bq.Query(
    sql, lat=float(settings.LATITUDE), lon=float(settings.LONGITUDE)
).to_dataframe()

In [6]:
stations.head()

,name,id,state,latitude,longitude,dist_kms
0,STUTTGART-STADT,GM000002716,None,48.7703,9.1825,0.742676
1,STUTTGART (NECKARTAL),GME00129118,None,48.7906,9.2178,4.291568
2,STUTTGART-SCHNARRENBERG,GME00115771,None,48.8292,9.2008,8.163890
3,STUTTGART-HOHENHEIM,GME00129143,None,48.7144,9.2103,9.348723
4,STUTTGART/ECHTERDINGEN,GME00111512,None,48.6892,9.2253,13.305016


In [7]:
# select stations within a radius of 30km around the selected Latitude and Longitude points
stations_stuttgart = stations[stations.dist_kms <= 30]

## Visualisation on Google Maps

In [8]:
# you don't necessarily need an API key, but it opens more options
google_map_api_key = settings.GOOGLE_MAP_API_KEY

# define the center of the map and zoom
google_map = gmplot.GoogleMapPlotter(48.7703, 9.1825, 10, apikey=google_map_api_key)

# prepare the data
locations = zip(
    *[
        (lat, long)
        for lat, long in zip(stations_stuttgart.latitude, stations_stuttgart.longitude)
    ]
)

# plot the dots on the map
google_map.scatter(*locations, color="red")

In [9]:
# plot the results on a html map
# google_map.draw("../results/map_weather_stations_stuttgart.html")

## Prepare the data

Filtering out the elements that are not meaningful for our analyses.

Here, we pull data from the `GM000002716` station for the dates of interest (daily rainfall precipitation for 2018). 

To get the rainfall amount ("precipitation" or PRCP) in millimeters, you’d write:

In [10]:
sql = utils.read_query("../data2day_2022/sql/extract_precipitation_2018.sql")

In [11]:
sql

"SELECT STRING(wx.date) AS date ,wx.value/10.0 AS prcp FROM [bigquery-public-data:ghcn_d.ghcnd_2018] AS wx WHERE id = $station_id AND qflag IS NULL AND element = 'PRCP' ORDER BY wx.date"

In [12]:
# GM000002716 is not in the table. GME00115771 is the first one for which it is the case.
weather_data_stuttgart_2018 = bq.Query(sql, station_id="GME00115771").to_dataframe()

In [13]:
weather_data_stuttgart_2018.head()

,date,prcp
0,2018-01-01,3.5
1,2018-01-02,0.6
2,2018-01-03,7.2
3,2018-01-04,12.4
4,2018-01-05,4.8


In [14]:
# save the result into a csv file
# weather_data_stuttgart_2018.to_csv("../results/weather_prcp.csv", encoding="UTF-8", index=False)